In [1]:
import numpy as np 
import py2neo as pn 
import random as rd
import timeit
import math
import copy

In [2]:
def barabasi(total_n, starting_n, starting_b):
    ### Generate a Barabasi-Albert network simulation
    # total_n: total number of nodes
    # starting_n: number of nodes to generate edges uniformly at random
    # returns subgraph: neo4j graph object
    ###
    edges = []
    nodes = [pn.Node('User', name='user' + str(i)) for i in range(starting_n)]
    for i in range(starting_n):
        # Generate links uniformly at random
        temp_list = list(range(starting_n))
        temp_list.remove(i)
        linkto = rd.sample(temp_list, starting_b)
        temp = [pn.Relationship(nodes[i], "follow", nodes[j]) for j in linkto]
        edges.extend(temp)
    for l in range(starting_n, total_n):
        nodes.append(pn.Node('User', name='user' + str(l)))
        n = len(edges)
        # Compute the probability for a new node to connect to each existing node
        temp_probs = [len(list(filter(lambda x: x.end_node == nodes[j] or x.start_node == nodes[j]
                                      , edges)))/(2*n) for j in range(len(nodes))]
        # Generate links for a new node
        temp = [pn.Relationship(nodes[l], "follow", nodes[j]) for j in range(l) 
                        if rd.uniform(0,1) < temp_probs[j] and l != j]
        edges.extend(temp)
    subgraph = pn.Subgraph(nodes,edges)
    return subgraph

In [3]:
def postpublish(graph, m0, m1):
    ### Simulate a list of posts and users who create them
    # graph: py2neo object for follower network
    # m0: starting number of posts
    # m1: total number of posts
    # returns postpublished and relationship subgraph
    ###
    postpublished = []
    relationships = []
    
    # Get the users and the number of users
    nodes = list(graph.run('MATCH (n: User) RETURN n'))
    n = len(nodes)
    
    # Generate the first posts
    for i in range(m0):
        new_post = pn.Node('Post', name='post' + str(i))
        postpublished.append(new_post)
        relationships.append(pn.Relationship(rd.choice(nodes)[0],'published',new_post))
    
    # Generate the rest of the posts
    for m in range(m0,m1):
        new_post = pn.Node('Post', name='post' + str(m))
        # Compute the prob that anyone posts
        probs = []
        for k in range(n):
            # Count number of followers
            temp_probs = list(graph.run('MATCH (n:User {name:"user' + str(3) + '"})<-[:follow]-(a) return count(a)').data()[0].values())[0]
            # Count number of posts
            temp_probs = temp_probs + len(list(filter(lambda x: x.start_node == nodes[k][0], relationships)))
            probs.append(temp_probs)
        # Choose a user for new post
        temp_user = rd.choices(nodes, weights = probs)
        postpublished.append(temp_user[0], 'published', new_post)  
    subgraph = pn.Subgraph(postpublished,relationships)
    return subgraph

In [4]:
def postview(P,N,graph):
    # depends on friends/followers/reads
    user_list = [pn.Node('User', name='user' + str(i)) for i in N]
    post_list = [pn.Node('Post', name='post' + str(i)) for i in P]
    view_list = [0] * len(P)
    rel_list = []
    k_all_friends = 0
    k_all_follow = 0
    for i in P:
        user = list(graph.run('match (n:Post{name:"post' + str(i) + '"})<-[:published]-(a) return a').data()[0].values())[0].nodes[0]['name']
        friends = list(graph.run('MATCH (n:User {name:"' + user + '"})-[:friends]-(a) return count(a)').data()[0].values())[0]
        k_all_friends += friends
        follow = list(graph.run('MATCH (n:User {name:"' + user + '"})<-[:follow]-(a) return count(a)').data()[0].values())[0]
        k_all_follow += follow
    for i in P:
        user = list(graph.run('match (n:Post{name:"post' + str(i) + '"})<-[:published]-(a) return a').data()[0].values())[0].nodes[0]['name']
        user_list_m = copy.deepcopy(user_list)
        for n in user_list_m:
            if n['name'] == user:
                user_list_m.remove(n)
        friends = list(graph.run('MATCH (n:User {name:"' + user + '"})-[:friends]-(a) return count(a)').data()[0].values())[0]
        follow = list(graph.run('MATCH (n:User {name:"' + user + '"})<-[:follow]-(a) return count(a)').data()[0].values())[0]
        p = (friends + follow + view_list[i]) / (k_all_friends + k_all_follow + sum(view_list))
        p_random = rd.random()
        if p_random <= p:
            user_choice = np.random.choice(user_list_m,1)[0]
            rel_list.append(pn.Relationship(user_choice,'viewed',post_list[i]))
            view_list[i] += 1
    s = pn.Subgraph(post_list,rel_list)
    return s

def postlike(P,N,graph):
    user_list = [pn.Node('User', name='user' + str(i)) for i in N]
    post_list = [pn.Node('Post', name='post' + str(i)) for i in P]
    liked_list = [0] * len(P)
    rel_list = []
    k_all_friends = 0
    k_all_follow = 0
    k_all_posted = 0
    for i in P:
        user = list(graph.run('match (n:Post{name:"post' + str(i) + '"})<-[:published]-(a) return a').data()[0].values())[0].nodes[0]['name']
        friends = list(graph.run('MATCH (n:User {name:"' + user + '"})-[:friends]-(a) return count(a)').data()[0].values())[0]
        k_all_friends += friends
        follow = list(graph.run('MATCH (n:User {name:"' + user + '"})<-[:follow]-(a) return count(a)').data()[0].values())[0]
        k_all_follow += follow
        post = list(graph.run('MATCH (n:User {name:"' + user + '"})-[:published]->(a) return count(a)').data()[0].values())[0] 
        k_all_posted += post
    for i in P:
        user = list(graph.run('match (n:Post{name:"post' + str(i) + '"})<-[:published]-(a) return a').data()[0].values())[0].nodes[0]['name']
        user_list_m = copy.deepcopy(user_list)
        for n in user_list_m:
            if n['name'] == user:
                user_list_m.remove(n)
        friends = list(graph.run('MATCH (n:User {name:"' + user + '"})-[:friends]-(a) return count(a)').data()[0].values())[0]
        follow = list(graph.run('MATCH (n:User {name:"' + user + '"})<-[:follow]-(a) return count(a)').data()[0].values())[0]
        post = list(graph.run('MATCH (n:User {name:"' + user + '"})-[:published]->(a) return count(a)').data()[0].values())[0]
        for j in N:
            p = (friends + follow + post + liked_list[i]) / (k_all_friends + k_all_follow + k_all_posted + sum(liked_list))
            p_random = rd.random()
            if p_random <= p:
                user_choice = np.random.choice(user_list_m,1)[0]
                rel_list.append(pn.Relationship(user_choice,'liked',post_list[i]))
                liked_list[i] += 1
    s = pn.Subgraph(post_list,rel_list)
    return s
    

In [5]:
# start a new project in Neo4j and set connections
graph = pn.Graph(
    host = 'localhost',
    http_port = '7474',
    user = 'neo4j',
    password = '2500'
    )   

In [ ]:
################ Experiment 1
start = timeit.default_timer()
NPosts = 1000
NUsers = 1000
P=list(range(500))
N=list(range(500))
graph.run('match (n:User) detach delete n')
graph.run('match (n:Post) detach delete n')
subgraph = barabasi(NUsers,10,5)
graph.merge(subgraph,'User','name')
subgraph_post = postpublish(graph,NPosts,10)
graph.merge(subgraph_post, 'User', 'name')
# stage 6
s_posts_viewed = postview(P,N,graph)
# stage 7
graph.merge(s_posts_viewed,'User','name')
# stage 8
s_posts_liked = postlike(P,N,graph)
# stage 9
graph.merge(s_posts_liked,'User','name')


stop = timeit.default_timer()

print('Time: ', stop - start)  

In [ ]:
# Store values for post liked and post viewed
exp1_posts_viewed = {}
exp1_posts_liked = {}
for i in P:
    exp1_posts_viewed['post'+str(i)]=list(graph.run('MATCH (n:Post {name:"post' + str(i) + '"})<-[:viewed]-(a) return count(a)').data()[0].values())[0]
    exp1_posts_liked['post'+str(i)]=list(graph.run('MATCH (n:Post {name:"post' + str(i) + '"})<-[:liked]-(a) return count(a)').data()[0].values())[0]

In [ ]:
exp1_posts_viewed

In [ ]:
list(graph.run('MATCH (n: User {name:"user' + str(3) + '"})<-[:follow]-(a: User {name:"user' + str(5) + '"}) return count(*)').data()[0].values())[0]

In [ ]:
list(graph.run('MATCH (n:User {name:"user' + str(3) + '"})<-[:follow]-(a) return count(a)').data()[0].values())[0]

In [ ]:
follow = 0
list(graph.run('MATCH (n: User) RETURN n'))

In [ ]:
list(graph.nodes)

In [ ]:
list(graph.run('MATCH (n:Post {name:"post' + str(76) + '"})<-[:liked]-(a) return count(a)').data()[0].values())[0]